### This code block works, however, returned values are limited to 100 per API limits.

In [62]:
import spotipy
import spotipy.util as util

scope = 'playlist-read-private'
cid ="5b14ab139b5b428abb0631807fbe753d" 
secret = "50cc15b5c98d412e8bb5d6f81e44696c"  
redirect = "http://localhost:8888/callback"
username='sdvzhr40fdjbtr1dp7mbnlfk4'
playlist= '1qKLFl2MHnP0RP8xiD26aB' #ID for 'likes' playlist.  Repeat for dislike once debugged.

token = util.prompt_for_user_token(username, scope=scope, client_id=cid, client_secret=secret, redirect_uri=redirect)

sp = spotipy.Spotify(auth=token)
playlist = sp.user_playlist(username, playlist)
songs = playlist['tracks']['items']
ids = []
for i in range(len(songs)):
    ids.append(songs[i]['track']['id'])
features = sp.audio_features(ids) #this is the function I'm trying to recreate down below

In [64]:
import pandas as pd
features = pd.DataFrame(features)
features.tail() #works but df limited to 100

,acousticness,analysis_url,danceability,duration_ms,energy,id,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,track_href,type,uri,valence
95,0.785,https://api.spotify.com/v1/audio-analysis/6oXm...,0.512,285387,0.434,6oXm8DyDZHtDxlq7SyOYSb,0.521000,0,0.0900,-12.853,1,0.0308,99.119,4,https://api.spotify.com/v1/tracks/6oXm8DyDZHtD...,audio_features,spotify:track:6oXm8DyDZHtDxlq7SyOYSb,0.4450
96,0.264,https://api.spotify.com/v1/audio-analysis/0rMA...,0.741,220216,0.488,0rMAjp5l2xRF6j19KXbtPe,0.000109,7,0.1060,-9.188,1,0.0405,134.024,4,https://api.spotify.com/v1/tracks/0rMAjp5l2xRF...,audio_features,spotify:track:0rMAjp5l2xRF6j19KXbtPe,0.5350
97,0.225,https://api.spotify.com/v1/audio-analysis/1Jsd...,0.597,296985,0.250,1JsdrqHHRz7RaSHxCgfuPE,0.084100,8,0.0823,-9.014,1,0.0313,106.889,4,https://api.spotify.com/v1/tracks/1JsdrqHHRz7R...,audio_features,spotify:track:1JsdrqHHRz7RaSHxCgfuPE,0.0618
98,0.608,https://api.spotify.com/v1/audio-analysis/6Ta8...,0.775,308961,0.540,6Ta8oLYe1iEBSnie0pDQoF,0.088200,7,0.0723,-7.619,1,0.0463,90.003,4,https://api.spotify.com/v1/tracks/6Ta8oLYe1iEB...,audio_features,spotify:track:6Ta8oLYe1iEBSnie0pDQoF,0.2670
99,0.316,https://api.spotify.com/v1/audio-analysis/7jDQ...,0.798,246856,0.812,7jDQBqpLFme3pUMRX31BsS,0.015800,9,0.1040,-7.259,0,0.0413,105.018,4,https://api.spotify.com/v1/tracks/7jDQBqpLFme3...,audio_features,spotify:track:7jDQBqpLFme3pUMRX31BsS,0.8390


### Below: attempts to iterate through using 'next' in client module http://spotipy.readthedocs.io/en/latest/#api-reference

In [65]:
import spotipy
import spotipy.util as util

scope = 'playlist-read-private'
cid ="5b14ab139b5b428abb0631807fbe753d" 
secret = "50cc15b5c98d412e8bb5d6f81e44696c"  
redirect = "http://localhost:8888/callback"
username='sdvzhr40fdjbtr1dp7mbnlfk4'
playlist= '1qKLFl2MHnP0RP8xiD26aB'

token = util.prompt_for_user_token(username, scope=scope, client_id=cid, client_secret=secret, redirect_uri=redirect)
sp = spotipy.Spotify(auth=token)

# function to extract track id from playlist
def get_playlist_tracks(username, playlist):
    playlist = sp.user_playlist_tracks(username,playlist)
    ids=[]
    
    while playlist['next']:
        playlist = sp.next(playlist)
        songs = playlist['items']
        for i in range(len(songs)):
            ids.append(songs[i]['track']['id'])
        return ids

# call function
ids = get_playlist_tracks(username, playlist)

# put into dataframe
df = pd.DataFrame(ids)
df.tail() 
#was mostly working earlier, returning 400+ values.  
#Should be returning 521, # of songs in playlist.  Now only 100.

,0
95,67bc0k8oAuIprA3X0xzWdQ
96,0x5EMnC2M9pDTAAStksrDt
97,2SPTGg9SC5MT1FwNX4IYfx
98,6Nfeou4uzrZW1jZ6KUHwai
99,42fGLwbmdlMvpNl1pl2axC


In [66]:
# use track id from above to get audio features
# I don't know how to write this function.  

def get_audio_features(ids):
    features = sp.audio_features(ids)
    audio_features = []
    while features['next']:
        features = sp.next(features)
        for i in ids:
            features.append(sp.audio_features(i))
    return audio_features

# call function
get_audio_features(ids)

TypeError: list indices must be integers or slices, not str

In [ ]:
# main spotipy functionality I'm relying on: 'sp.get_playlist_tracks' and 'sp.audio_features'.  
# the rest is login information and attempts to iterate